In [1]:
# Import tensorflow package
import tensorflow as tf

# Python Tutorial - TensorFlow

TensorFlow works with graphs and sessions. First we build our model (i.e the graph). Nothing is being calculated at that point. Then after our model is ready (i.e the graph) we run the session in order to actually run the model!


![title](../img/tensorflow_graphs.png)


Now, we will introduce some of the basic objects and operations, and also the concept of a session and a graph!

## 1. Constants (& Sessions)


First we introduce the constant. After we define it, we cannot change the value!

In [2]:
# A constant
a = tf.constant([[3, 6],],                   # The value of the constant (here a 1x2 row vector)
                name='a',                    # The name of the constant so it can be used to the graph
                shape = (1,2),               # The shape of the constant (if it's scalar, vector, matrix)
                verify_shape=True)           # If True and value is not of the right shape, it throws an error. 
                                             # If False and value is not of the right shape, broadcastes to bring it to bring it to the right form.
# Another constant
b = tf.constant([[2, 2],],
                name='b', 
                shape = (1,2), 
                verify_shape=True)

# Addition of two constants.
addition = tf.add(a,b, name='addition')                                             # Element wise addition >> [5 8]

# Elemtn wise multiplication of two constants. 
multiplication = tf.multiply(a,b, name='multiplication')                            # Element wise multiplication >> [6 12]

# Matrix Multiplication of two constants.
matrix_multiplication = tf.matmul(tf.reshape(a, [1, 2]), tf.reshape(b, [2, 1]))     # Matrix multiplication >> [[18]]

# Division 
division = tf.div(a, b)                                                             # Element wise division >> [1 3]

# Modulo
modulo = tf.mod(a, b)                                                               # Element wise modulo >> [1 0]

# Power x^y
power = tf.pow(a,b, name='power')                                                   # Element wise power >> [9 36]
 
# We can create more complex objects with results of previous operations.
operation1 = tf.pow(addition,multiplication, name='operation1')


A Session object encapsulates the environment in which Operation objects are
executed, and Tensor objects are evaluated.

In [3]:
# We create a session and we run all the operations we created! Now, we actually have the results!
with tf.Session() as sess:
    add, mul, matmul, div, mod, pw, op1 = sess.run(fetches = [addition, multiplication, matrix_multiplication, division, modulo, power, operation1])
    print(add, mul, matmul, div, mod, pw, op1)

[[5 8]] [[ 6 12]] [[18]] [[1 3]] [[1 0]] [[ 9 36]] [[15625     0]]


A session can also calculate specific nodes. It will automatically take into account only the nodes that are needed in order to calculate the node and it will neglect all the useless ones.

Some more useful for NN's constants!

In [88]:
# A tensor of given dimensions full with 0's and 1's!
zeros = tf.zeros((2, 3))       # A 2x3 matrix full of zeros
ones = tf.ones((3,2))          # A 3x2 matrix full of ones

# In general form (works with any number)
zeros = tf.fill(dims = (2, 3), value = 0)
ones = tf.fill(dims = (3, 2), value = 1)

# Given a tensor x, we can create a tensor of same dimensions full of 0's or 1's
zeros = tf.zeros_like(x)
ones = tf.ones_like(x)

# Range, LIMiT IS NOT INCLUDED (same as python and numpy)
ran = tf.range(start = 3, limit=18, delta=3)

# Sequence of numbers (same as python and numpy)
linspace = tf.linspace(start = 10.0, stop = 13.0, num = 4)

# Randomly Generated Constants (Many more, here only 3 examples)
normal = tf.random_normal(shape, mean=0.0, stddev=1.0)                # Normal Distribution
unifrom = tf.random_uniform(shape, minval=0, maxval=None)             # Unifrom Distribution
shuffle = tf.random_shuffle(value, seed=None, name=None)              # Shuffling the values of a tensor (only Batch dimension)

## 2. Variables (& Sessions)

Now, we introduce the variable. After we define it we can change the value in the future. Same operations as in constants, but Variable is now a Class, and there are a lot of methods we can apply onto it!

**REMEMBER** that we alwasy need to initialize variables before run the session.

In [65]:
# A Sclarar Variable
a = tf.Variable(2, name="scalar")

# A Vector Variable
b = tf.Variable([2, 3], name="vector")

# A Matrix Variable
c = tf.Variable([[0, 1], [2, 3]], name="matrix")

# Assign Operation
assign_op = a.assign(2*a + 10)    # We create an opereation, in which we assign a new value to a.
                                  # BE CAREFUL: WE NEED TO RUN IT IN ORDER TO HAPPEN. OTHERWISE IT WILL BE ONLY DEFINED!


Now, we initialize the variables, and then we execute a session!

In [66]:
# Let's initialize a and run the session
with tf.Session() as sess:    
    sess.run(a.initializer)                 # We initialize the variable! The assign operation has not run yet!
    print(a.eval())                         # eval() is a method applied on Variables, which shows the actual tensor!
    
    sess.run(assign_op)                     # Now we run the assign operation! The values of a changes according to assign operation
    print(a.eval())
    
    
# The easiest way is initializing all variables at once:
init = tf.global_variables_initializer()    # Initialize all variables
with tf.Session() as sess:
    sess.run(init)
    print(a.eval(), b.eval(), c.eval())     # Here we have not run the assign_op. So here a is still the initial one!

2
14
2 [2 3] [[0 1]
 [2 3]]


## 3. Placeholders (& Sessions)

Now, we introduce the placeholder. Placeholders are like empty variables, that we can fill in later. We can have them empty, and we can fill them in during execution. It's like defining a function $f(x, y)$, without knowing value of x or y. x, y are placeholders for the actual values.

In [102]:
# A placeholder of type float 32-bit, shape is a vector of 3 elements
a = tf.placeholder(tf.float32, shape=[3], name='a')

# Another placeholder
b = tf.placeholder(tf.float32, shape=[2,1], name ='b')

If we want to run a session, we have to assign values to the placeholders. Otherwise we will get an error. The way to assign values is through a dictionary like it follows:

In [105]:
with tf.Session() as sess:
    
    feed_dict = {a:[1,2,3], b:[[1],[2]]}             # Define the values
    print(sess.run([a,b], feed_dict=feed_dict))      # Run the session with these values  
    
    for i in list(range(5)):                         # Or for multiple evaluations (e.g for differenct epochs)
        print(sess.run(a, feed_dict={a:[i,i,i]})) 

[array([ 1.,  2.,  3.], dtype=float32), array([[ 1.],
       [ 2.]], dtype=float32)]
[ 0.  0.  0.]
[ 1.  1.  1.]
[ 2.  2.  2.]
[ 3.  3.  3.]
[ 4.  4.  4.]


## 4. Graphs

Graphs help to visualize the network we built. After we create a graph we can lunch the serven using:
<br><br>
$$\text{tensorboard --logdir="./graphs" --port 6006}$$ 
<br>
The graphs can be found on http://localhost:6006/

In [55]:
# Clean the graph from everything (otherwise it keeps the old nodes)
tf.reset_default_graph()

# Create a computational graph
x = tf.constant(1, name='x')  
y = tf.constant(3, name='y')
addition = tf.add(x,y, name='addition')

# Create a visual graph
writer = tf.summary.FileWriter('./graphs', sess.graph)

# Run the session (now a graph will also be created)
with tf.Session() as sess:
    sess.run(addition)
